# Setup

Neste bloco é realizado a importação de bibliotecas tanto para base de dados quanto para elaboração do modelo.

In [92]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importar base de dados

In [85]:
# Carregar dados
cifar10 = tf.keras.datasets.cifar10

# Distribuir dados em treino e teste
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [86]:
# Normalizar valores
x_train, x_test = x_train / 255.0, x_test / 255.0

# Converter labels
y_train, y_test = y_train.flatten(), y_test.flatten()

# Data Augmentation

In [87]:
# Configurar Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.1,
    height_shift_range=0.1,
    width_shift_range=0.1
)

# Aplicar o augmentation durante o treinamento
train_generator = datagen.flow(x_train, y_train, 32)

In [89]:
x_train[0].shape

(32, 32, 3)

# Modelo

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Dense

# Criação do modelo sequencial
model = Sequential()

# Adicionando camadas
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))

# Camada oculta
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

# Camada de output
model.add(Dense(10, activation='softmax'))

# Descrição do modelo
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_30 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_31 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_31 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                      

In [69]:
# Compilação do modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Treinamento

In [70]:
# Definir Early Stopping
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [71]:
# Treinar modelo
history = model.fit(train_generator, validation_data=(x_test, y_test), epochs=50, callbacks=[early_stopping])

Epoch 1/50
1563/1563 [==============================] - 50s 28ms/step - loss: 1.5296 - accuracy: 0.4650 - val_loss: 1.4159 - val_accuracy: 0.5297
Epoch 2/50
1563/1563 [==============================] - 40s 26ms/step - loss: 1.1070 - accuracy: 0.6122 - val_loss: 0.9853 - val_accuracy: 0.6667
Epoch 3/50
1563/1563 [==============================] - 40s 26ms/step - loss: 0.9506 - accuracy: 0.6712 - val_loss: 0.9151 - val_accuracy: 0.7029
Epoch 4/50
1563/1563 [==============================] - 41s 26ms/step - loss: 0.8573 - accuracy: 0.7033 - val_loss: 0.8478 - val_accuracy: 0.7099
Epoch 5/50
1563/1563 [==============================] - 42s 27ms/step - loss: 0.7957 - accuracy: 0.7284 - val_loss: 0.7487 - val_accuracy: 0.7516
Epoch 6/50
1563/1563 [==============================] - 41s 26ms/step - loss: 0.7329 - accuracy: 0.7498 - val_loss: 0.7773 - val_accuracy: 0.7529
Epoch 7/50
1563/1563 [==============================] - 40s 26ms/step - loss: 0.6824 - accuracy: 0.7665 - val_loss: 0.6961 -

In [93]:
import plotly.graph_objects as go

# Dados de precisão do histórico de treinamento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

# Criando o gráfico
fig = go.Figure()

# Adicionando a linha de precisão do treinamento
fig.add_trace(go.Scatter(x=list(epochs), y=acc, mode='lines', name='Acurácia Treinamento'))

# Adicionando a linha de precisão da validação
fig.add_trace(go.Scatter(x=list(epochs), y=val_acc, mode='lines', name='Acurácia Validação'))

# Configurando o layout do gráfico
fig.update_layout(
    title='Acurácia por Iteração',
    xaxis_title='Épocas',
    yaxis_title='Acurácia',
    legend_title='Tipo de Acurácia'
)

# Exibindo o gráfico


# Salvar modelo

In [79]:
# Salve o modelo
model.save('cifarmodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [80]:
# Baixe o modelo para o seu computador
from google.colab import files
files.download('cifarmodel.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>